In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import sklearn
import matplotlib.pyplot as plt
import torchvision

In [2]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels,kernel_size=3, stride= stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x  = x + identity
        x = self.relu(x)
        return x

In [3]:
class Resnet(nn.Module): # layers list of how many times we are going to use the block
    def __init__(self, block, layers, image_channels, num_classes):
        super(Resnet, self).__init__()
        # Steam
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # ResNet Layers
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride= 1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride= 2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride= 2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride= 2) # 2048
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x
    
    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        if stride !=1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1
                                                         , stride=stride),
                                               nn.BatchNorm2d(out_channels*4))
        
        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4
        for i in range(num_residual_blocks-1):
            layers.append(block(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)
        

In [4]:
def Resnet50(in_channels=3, num_classes=1000):
    return Resnet(block, [3,4,6,3], in_channels, num_classes)

def Resnet101(in_channels=3, num_classses=1000):
    return Resnet(block, [3,4,23,3], in_channels, num_classes)

def Resnet152(in_channels=3, num_classses=1000):
    return Resnet(block, [3,8,36,3], in_channels, num_classes)

In [5]:
def test():
    net = Resnet50()
    x = torch.randn(2, 3, 224, 224)
    y = net(x).to('cpu')
    print(y.shape)

In [6]:
test()

torch.Size([2, 1000])


In [7]:
imgSize = 224

# Transformations
# Training transformer
transformer = transforms.Compose([
    transforms.RandomRotation(degrees = 20),
    
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.RandomVerticalFlip(p = 0.3),
    
    transforms.Resize(size = (imgSize, imgSize), antialias = True),
    transforms.ToTensor(),
#     transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

# Validation transformer
valTransformer = transforms.Compose([
    transforms.Resize(size = (imgSize, imgSize), antialias = True),
    transforms.ToTensor(),
#     transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [8]:
# hyperparameters
BATCH_SIZE = 32
learning_rate = 0.0001

In [9]:
path = '/kaggle/input/melanoma-cancer-dataset/train'
valPath = '/kaggle/input/melanoma-cancer-dataset/test'

# Reading data
trainData = datasets.ImageFolder(root = path, transform = transformer)
testData = datasets.ImageFolder(root = valPath, transform = valTransformer)

train_loader = DataLoader(dataset=trainData,batch_size= BATCH_SIZE, shuffle= True)
test_loader = DataLoader(dataset=testData, batch_size= BATCH_SIZE, shuffle= False)

In [10]:
device= 'cuda'
model = Resnet50()
model.to(device)

Resnet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [12]:
def check_accuracy(loader, model, Train= True):
    if Train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    acc = float(num_correct/float(num_samples))
    print(f'{num_correct}/{num_samples} with accuracy {float(num_correct/float(num_samples))*100:.2f}')
    model.train()
    return acc

In [13]:
# Early Stopping
patience = 5
minDelta = 0.01
currentPatience = 0
bestLoss = float('inf')

In [14]:
epochs = 10
for epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(enumerate(train_loader), total= len(train_loader), leave= False)
    for batch_idx, (data, targets) in loop:
        # Send data to device
        data = data.to(device)
        targets = targets.to(device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        
        #log
        runningLoss += loss.item()
        
        #otimizer
        optimizer.step()
        
        # update progress bar
        loop.set_description(f'Epoch [{epoch}/{epochs}]')
        loop.set_postfix(loss = loss.item())
    
    trainLoss= runningLoss/len(train_loader)
    check_accuracy(train_loader, model, Train=True)
check_accuracy(test_loader, model, Train=False)

Checking accuracy on training data
10014/11879 with accuracy 84.30


Checking accuracy on training data
9675/11879 with accuracy 81.45


Checking accuracy on training data
10314/11879 with accuracy 86.83


Checking accuracy on training data
7777/11879 with accuracy 65.47


Checking accuracy on training data
10423/11879 with accuracy 87.74


...

Checking accuracy on training data
10557/11879 with accuracy 88.87


Checking accuracy on training data
9905/11879 with accuracy 83.38


Checking accuracy on training data
10548/11879 with accuracy 88.80


Checking accuracy on training data
10599/11879 with accuracy 89.22
Checking accuracy on test data
1841/2000 with accuracy 92.05


0.9205000400543213